<a href="https://colab.research.google.com/github/AbhinandanRoy7/Project/blob/main/FraudDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
!unzip /content/gdrive/MyDrive/Fraud.csv

Archive:  /content/gdrive/MyDrive/Fraud.csv
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of /content/gdrive/MyDrive/Fraud.csv or
        /content/gdrive/MyDrive/Fraud.csv.zip, and cannot find /content/gdrive/MyDrive/Fraud.csv.ZIP, period.


In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import IsolationForest
from scipy.stats import zscore
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [2]:
df = pd.read_csv('/content/gdrive/MyDrive/Fraud.csv')

In [3]:
df.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6362620 entries, 0 to 6362619
Data columns (total 11 columns):
 #   Column          Dtype  
---  ------          -----  
 0   step            int64  
 1   type            object 
 2   amount          float64
 3   nameOrig        object 
 4   oldbalanceOrg   float64
 5   newbalanceOrig  float64
 6   nameDest        object 
 7   oldbalanceDest  float64
 8   newbalanceDest  float64
 9   isFraud         int64  
 10  isFlaggedFraud  int64  
dtypes: float64(5), int64(3), object(3)
memory usage: 534.0+ MB


In [5]:
df.describe()

,step,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
count,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06
mean,2.433972e+02,1.798619e+05,8.338831e+05,8.551137e+05,1.100702e+06,1.224996e+06,1.290820e-03,2.514687e-06
std,1.423320e+02,6.038582e+05,2.888243e+06,2.924049e+06,3.399180e+06,3.674129e+06,3.590480e-02,1.585775e-03
min,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.560000e+02,1.338957e+04,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,2.390000e+02,7.487194e+04,1.420800e+04,0.000000e+00,1.327057e+05,2.146614e+05,0.000000e+00,0.000000e+00
75%,3.350000e+02,2.087215e+05,1.073152e+05,1.442584e+05,9.430367e+05,1.111909e+06,0.000000e+00,0.000000e+00
max,7.430000e+02,9.244552e+07,5.958504e+07,4.958504e+07,3.560159e+08,3.561793e+08,1.000000e+00,1.000000e+00


In [6]:
print(f"Rows: {df.shape[0]}, Columns: {df.shape[1]}")

Rows: 6362620, Columns: 11


In [7]:
z_scores = np.abs(zscore(df.select_dtypes(include=[np.number])))  # Only numeric cols
df_no_outliers = df[(z_scores < 3).all(axis=1)]  # Remove rows with z-score > 3

In [8]:
# Outlier Detection using Isolation Forest ---
iso_forest = IsolationForest(contamination=0.01, random_state=42)
outlier_pred = iso_forest.fit_predict(df.select_dtypes(include=[np.number]))
df = df[outlier_pred == 1]  # Keep only non-outliers

In [9]:
# Checking Multicollinearity using VIF
from statsmodels.stats.outliers_influence import variance_inflation_factor
import numpy as np

# Ensure All Columns Are Numeric
X_vif = df.select_dtypes(include=[np.number])  # Keep only numerical features

# Check for NaN or Infinite Values
X_vif = X_vif.replace([np.inf, -np.inf], np.nan)  # Replace infinities with NaN
X_vif = X_vif.dropna()  # Drop rows with NaN

# Compute VIF
vif_data = pd.DataFrame()
vif_data["feature"] = X_vif.columns
vif_data["VIF"] = [variance_inflation_factor(X_vif.values, i) for i in range(X_vif.shape[1])]

# Remove Highly Correlated Features (But Keep Key Features)
important_features = ["oldbalanceOrg", "newbalanceOrig", "oldbalanceDest", "newbalanceDest"]
features_to_remove = vif_data[(vif_data["VIF"] > 5) & (~vif_data["feature"].isin(important_features))]["feature"].tolist()

df = df.drop(columns=features_to_remove)

print("Dropped Features:", features_to_remove)


Dropped Features: []


In [10]:
df.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


In [11]:
df['isFraud'].value_counts(normalize=True)

,proportion
isFraud,
0,0.998827
1,0.001173


In [12]:
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Encode 'type' column using Label Encoding
label_encoder = LabelEncoder()
df["type"] = label_encoder.fit_transform(df["type"])

# Create Fraud-Based Features Without Removing 'nameOrig' and 'nameDest'
df["nameOrig_tx_count"] = df.groupby("nameOrig")["nameOrig"].transform("count")
df["nameDest_tx_count"] = df.groupby("nameDest")["nameDest"].transform("count")

# Encode 'nameOrig' and 'nameDest' using Label Encoding (Instead of Dropping)
df["nameOrig_encoded"] = label_encoder.fit_transform(df["nameOrig"])
df["nameDest_encoded"] = label_encoder.fit_transform(df["nameDest"])

X = df.drop(["isFraud", "nameOrig", "nameDest"], axis=1)  # Features (Only numeric now)
y = df["isFraud"]  # Target variable

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Apply SMOTE (Oversampling Fraud Cases)
smote = SMOTE(sampling_strategy=0.3, random_state=42)  # Adjust ratio
X_train_sm, y_train_sm = smote.fit_resample(X_train, y_train)

# Apply Undersampling (Reduce Majority Class)
undersample = RandomUnderSampler(sampling_strategy=0.7, random_state=42)
X_train_final, y_train_final = undersample.fit_resample(X_train_sm, y_train_sm)

# Check Class Balance After SMOTE
print("Before SMOTE:\n", y_train.value_counts(normalize=True))
print("After SMOTE:\n", y_train_final.value_counts(normalize=True))


Before SMOTE:
 isFraud
0    0.998827
1    0.001173
Name: proportion, dtype: float64
After SMOTE:
 isFraud
0    0.588235
1    0.411765
Name: proportion, dtype: float64


In [13]:
df.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud,nameOrig_tx_count,nameDest_tx_count,nameOrig_encoded,nameDest_encoded
0,1,3,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0,1,1,750182,1661298
1,1,3,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0,1,1,2167117,1733127
2,1,4,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0,1,42,991986,439069
3,1,1,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0,1,40,5770180,391150
4,1,3,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0,1,1,3411737,828129


In [15]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Encode the 'type' column using Label Encoding
label_encoder = LabelEncoder()
df["type"] = label_encoder.fit_transform(df["type"])

# Create New Features from 'nameOrig' and 'nameDest'
df["nameOrig_count"] = df.groupby("nameOrig")["nameOrig"].transform("count")
df["nameDest_count"] = df.groupby("nameDest")["nameDest"].transform("count")

nameOrig_freq = df["nameOrig"].value_counts().to_dict()
nameDest_freq = df["nameDest"].value_counts().to_dict()

df["nameOrig_freq"] = df["nameOrig"].map(nameOrig_freq)
df["nameDest_freq"] = df["nameDest"].map(nameDest_freq)

df = df.drop(["nameOrig", "nameDest"], axis=1)

X = df.drop(["isFraud"], axis=1)  # Features
y = df["isFraud"]  # Target Variable

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#Apply SMOTE for Balancing Data
smote = SMOTE(sampling_strategy="auto", random_state=42)
X_train_sm, y_train_sm = smote.fit_resample(X_train, y_train)

print("Before SMOTE:\n", y_train.value_counts())
print("After SMOTE:\n", y_train_sm.value_counts())


Before SMOTE:
 isFraud
0    5033301
1       5896
Name: count, dtype: int64
After SMOTE:
 isFraud
0    5033301
1    5033301
Name: count, dtype: int64


In [20]:
df.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud,nameOrig_tx_count,nameDest_tx_count,nameOrig_encoded,nameDest_encoded
0,1,3,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0,1,1,750182,1661298
1,1,3,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0,1,1,2167117,1733127
2,1,4,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0,1,42,991986,439069
3,1,1,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0,1,40,5770180,391150
4,1,3,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0,1,1,3411737,828129


In [23]:
### Feature Engineering ###

# Log Transformation for 'amount' (to reduce skewness)
df['log_amount'] = np.log1p(df['amount'])  # log(1 + amount) to avoid log(0)

df['orig_balance_change'] = (df['oldbalanceOrg'] - df['newbalanceOrig']) / (df['oldbalanceOrg'] + 1)
df['dest_balance_change'] = (df['oldbalanceDest'] - df['newbalanceDest']) / (df['oldbalanceDest'] + 1)

df['type_encoded'] = df['type'].astype('category').cat.codes

df['nameOrig_hash'] = df['nameOrig'].apply(lambda x: hash(x) % 10**6)  # Hash encoding
df['nameDest_hash'] = df['nameDest'].apply(lambda x: hash(x) % 10**6)

df.drop(columns=['amount', 'type', 'nameOrig', 'nameDest'], inplace=True)

# Convert to save memory (important for large datasets)
for col in df.columns:
    df[col] = df[col].astype(np.float32)

print(df.info())
df.head()


<class 'pandas.core.frame.DataFrame'>
Index: 6298997 entries, 0 to 6362619
Data columns (total 17 columns):
 #   Column               Dtype  
---  ------               -----  
 0   step                 float32
 1   oldbalanceOrg        float32
 2   newbalanceOrig       float32
 3   oldbalanceDest       float32
 4   newbalanceDest       float32
 5   isFraud              float32
 6   isFlaggedFraud       float32
 7   nameOrig_tx_count    float32
 8   nameDest_tx_count    float32
 9   nameOrig_encoded     float32
 10  nameDest_encoded     float32
 11  log_amount           float32
 12  orig_balance_change  float32
 13  dest_balance_change  float32
 14  type_encoded         float32
 15  nameOrig_hash        float32
 16  nameDest_hash        float32
dtypes: float32(17)
memory usage: 456.5 MB
None


,step,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud,nameOrig_tx_count,nameDest_tx_count,nameOrig_encoded,nameDest_encoded,log_amount,orig_balance_change,dest_balance_change,type_encoded,nameOrig_hash,nameDest_hash
0,1.0,170136.0,160296.359375,0.0,0.0,0.0,0.0,1.0,1.0,750182.0,1661298.0,9.194276,0.057834,0.000000,3.0,481512.0,699329.0
1,1.0,21249.0,19384.720703,0.0,0.0,0.0,0.0,1.0,1.0,2167117.0,1733127.0,7.531167,0.087731,0.000000,3.0,167874.0,157322.0
2,1.0,181.0,0.000000,0.0,0.0,1.0,0.0,1.0,42.0,991986.0,439069.0,5.204007,0.994505,0.000000,4.0,400574.0,278891.0
3,1.0,181.0,0.000000,21182.0,0.0,1.0,0.0,1.0,40.0,5770180.0,391150.0,5.204007,0.994505,0.999953,1.0,405678.0,724540.0
4,1.0,41554.0,29885.859375,0.0,0.0,0.0,0.0,1.0,1.0,3411737.0,828129.0,9.364703,0.280788,0.000000,3.0,819072.0,787095.0


In [16]:
!pip install catboost

In [17]:
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, recall_score

X = df.drop(columns=["isFraud"])  # Features
y = df["isFraud"]  # Target variable

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

catboost_model = CatBoostClassifier(iterations=1000, learning_rate=0.1, depth=8, loss_function='Logloss', eval_metric='AUC', verbose=200)

In [25]:
# Train the model
catboost_model.fit(X_train, y_train, eval_set=(X_test, y_test), early_stopping_rounds=1)

0:	test: 0.8646255	best: 0.8646255 (0)	total: 1.65s	remaining: 27m 30s
Stopped by overfitting detector  (1 iterations wait)

bestTest = 0.9670429481
bestIteration = 2

Shrink model to first 3 iterations.


In [26]:
# Predictions
y_pred = catboost_model.predict(X_test)
y_pred_proba = catboost_model.predict_proba(X_test)[:, 1]

In [27]:
# Model Evaluation
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_proba)
recall = recall_score(y_test, y_pred)

print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"ROC-AUC Score: {roc_auc:.4f}")
print(f"Recall: {recall:.4f}")

Accuracy: 0.9992
F1 Score: 0.4945
ROC-AUC Score: 0.9670
Recall: 0.3324
